---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [59]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [60]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [63]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    with open('university_towns.txt') as file:
        data = []
        for line in file:
            data.append(line[:-1])
            
        state_town = []
        for line in data:
            if line[-6:] == '[edit]':
                state = line[:-6]
            elif '(' in line:
                town = line[:line.index('(')-1]
                state_town.append([state,town])
            else:
                town = line.rstrip('')
                state_town.append([state,town])
    
    return pd.DataFrame(state_town, columns = ['State', 'RegionName'])
#get_list_of_university_towns()

In [64]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdp = pd.read_excel('gdplev.xls',skiprows = 7)
    gdp = gdp[gdp.columns[[4, 5, 6]]]
    gdp.columns = ['Quarter','GDP Current', 'GDP 2009']
    gdp = gdp.iloc[212:]
    gdp = gdp.reset_index()
    gdp = gdp.drop('index' ,axis = 1)
    for i in range(len(gdp)-4):
        if gdp.iloc[i][1] > gdp.iloc[i+1][1] and gdp.iloc[i+1][1] > gdp.iloc[i+2][1]:
            recession_start = gdp.iloc[i][0]
            break
        
    return recession_start
#get_recession_start()

In [65]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    gdp = pd.read_excel('gdplev.xls',skiprows = 7)
    gdp = gdp[gdp.columns[[4, 5, 6]]]
    gdp.columns = ['Quarter','GDP Current', 'GDP 2009']
    gdp = gdp.iloc[212:]
    gdp = gdp.reset_index()
    gdp = gdp.drop('index' ,axis = 1)
    for i in range(len(gdp)-4):
        if ((gdp.iloc[i][1] > gdp.iloc[i+1][1]) & (gdp.iloc[i+1][1] > gdp.iloc[i+2][1]) & (gdp.iloc[i+2][1] < gdp.iloc[i+3][1]) & (gdp.iloc[i+3][1] < gdp.iloc[i+4][1])):
            recession_end = gdp.iloc[i+4][0]
            break
        
    return recession_end

#get_recession_end()

In [66]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',skiprows = 7)
    gdp = gdp[gdp.columns[[4, 5, 6]]]
    gdp.columns = ['Quarter','GDP Current', 'GDP 2009']
    gdp = gdp.iloc[212:]
    gdp = gdp.reset_index()
    gdp = gdp.drop('index' ,axis = 1)
    for i in range(len(gdp)-4):
        if ((gdp.iloc[i][1] > gdp.iloc[i+1][1]) & (gdp.iloc[i+1][1] > gdp.iloc[i+2][1]) & (gdp.iloc[i+2][1] < gdp.iloc[i+3][1]) & (gdp.iloc[i+3][1] < gdp.iloc[i+4][1])):
            recession_bottom = gdp.iloc[i+2][0]
            break
        
    return recession_bottom

#get_recession_bottom()

In [67]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    house_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    house_df = house_df.drop(house_df.columns[[0] + list(range(3,51))], axis=1)
    house_df['State'] = house_df['State'].apply(lambda x : states[x])
    ans = house_df[['State','RegionName']]
    
    for year in range(2000,2016):
        ans[str(year) + 'q1'] = house_df[[str(year) + '-01', str(year) + '-02', str(year) + '-03']].mean(axis = 1)
        ans[str(year) + 'q2'] = house_df[[str(year) + '-04', str(year) + '-05', str(year) + '-06']].mean(axis = 1)
        ans[str(year) + 'q3'] = house_df[[str(year) + '-07', str(year) + '-08', str(year) + '-09']].mean(axis = 1)
        ans[str(year) + 'q4'] = house_df[[str(year) + '-10', str(year) + '-11', str(year) + '-12']].mean(axis = 1)
    
    year = 2016
    ans[str(year) + 'q1'] = house_df[[str(year) + '-01', str(year) + '-02', str(year) + '-03']].mean(axis = 1)
    ans[str(year) + 'q2'] = house_df[[str(year) + '-04', str(year) + '-05', str(year) + '-06']].mean(axis = 1)
    ans[str(year) + 'q3'] = house_df[[str(year) + '-07', str(year) + '-08']].mean(axis = 1)
    
    ans = ans.set_index(['State','RegionName'])
    
    return ans

#convert_housing_data_to_quarters()

In [68]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    towns = get_list_of_university_towns()
    bottom = get_recession_bottom()
    start = get_recession_start()
    houses = convert_housing_data_to_quarters()
    start = houses.columns[houses.columns.get_loc(start) - 1]
    houses['Change'] = houses[start] - houses[bottom]
    houses = houses[[start, bottom, 'Change']]
    houses = houses.reset_index()
    
    df = pd.merge(houses, towns, how = 'inner', on = ['State','RegionName'])
    df['University'] = True
    
    df2 = pd.merge(houses, df, how = 'outer', on = ['State','RegionName', start, bottom, 'Change'])
    df2['University'] = df2['University'].fillna(False)
    
    Uni = df2[df2['University'] == True]
    Non_Uni = df2[df2['University'] == False]
    
    tValue, pValue = ttest_ind(Uni['Change'].dropna(), Non_Uni['Change'].dropna())
    
    difference = pValue < 0.01
    better = "university town" if Uni['Change'].mean() < Non_Uni['Change'].mean() else "non-university town"
    
    return (difference, pValue, better)

#run_ttest()